In [2]:
import pickle
a_auc_map = pickle.load(open('a_auc_map', 'rb'))
h_auc_map = pickle.load(open('h_auc_map', 'rb'))

In [3]:
print(a_auc_map.keys())

dict_keys(['defect_camel_1_2', 'defect_eclipse_2_0', 'defect_eclipse_3_0', 'defect_prop_2', 'defect_xalan_2_6'])


In [4]:
print(h_auc_map.keys())

dict_keys(['H_eclipse_2_0_xgboost', 'H_eclipse_2_0_randomforest', 'H_eclipse_2_0_lr', 'H_camel_1_2_xgboost', 'H_camel_1_2_randomforest', 'H_camel_1_2_lr', 'H_eclipse_3_0_xgboost', 'H_eclipse_3_0_randomforest', 'H_eclipse_3_0_lr', 'H_prop_2_xgboost', 'H_prop_2_randomforest', 'H_prop_2_lr', 'H_xalan_2_6_xgboost', 'H_xalan_2_6_randomforest', 'H_xalan_2_6_lr'])


In [10]:
import scipy.stats

In [20]:
wil_pval=[]
for key in a_auc_map.keys():
    kk = key.split("defect_")[1]
    print(kk)
    for hkey in h_auc_map.keys():
        if kk in hkey:
            print(hkey)
            aa = a_auc_map[key]
            hh = h_auc_map[hkey]
            wil_test = scipy.stats.wilcoxon(aa,hh)
            print(wil_test)
            wil_pval.append(wil_test.pvalue)

camel_1_2
H_camel_1_2_xgboost
WilcoxonResult(statistic=4.0, pvalue=6.436988241970729e-18)
H_camel_1_2_randomforest
WilcoxonResult(statistic=1165.0, pvalue=4.824465542235559e-06)
H_camel_1_2_lr
WilcoxonResult(statistic=1648.0, pvalue=0.0038959032430327762)
eclipse_2_0
H_eclipse_2_0_xgboost
WilcoxonResult(statistic=2.0, pvalue=6.055964751718375e-18)
H_eclipse_2_0_randomforest
WilcoxonResult(statistic=1.0, pvalue=5.873890051096796e-18)
H_eclipse_2_0_lr
WilcoxonResult(statistic=0.0, pvalue=5.697220948724935e-18)
eclipse_3_0
H_eclipse_3_0_xgboost
WilcoxonResult(statistic=0.0, pvalue=5.697220948724935e-18)
H_eclipse_3_0_randomforest
WilcoxonResult(statistic=15.0, pvalue=8.996130287159537e-18)
H_eclipse_3_0_lr
WilcoxonResult(statistic=1.0, pvalue=5.873890051096796e-18)
prop_2
H_prop_2_xgboost
WilcoxonResult(statistic=1613.0, pvalue=0.0026242868364230896)
H_prop_2_randomforest
WilcoxonResult(statistic=0.0, pvalue=5.697220948724935e-18)
H_prop_2_lr
WilcoxonResult(statistic=0.0, pvalue=5.6972209

In [48]:
from statsmodels.stats import multitest 
multitest.multipletests(pvals=wil_pval,method='holm')[1]<0.05

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [37]:

def cliffsDelta(lst1, lst2, **dull):

    """Returns delta and true if there are more than 'dull' differences"""
    if not dull:
        dull = {'small': 0.147, 'medium': 0.33, 'large': 0.474} # effect sizes from (Hess and Kromrey, 2004)
    m, n = len(lst1), len(lst2)
    lst2 = sorted(lst2)
    j = more = less = 0
    for repeats, x in runs(sorted(lst1)):
        while j <= (n - 1) and lst2[j] < x:
            j += 1
        more += j*repeats
        while j <= (n - 1) and lst2[j] == x:
            j += 1
        less += (n - j)*repeats
    d = (more - less) / (m*n)
    size = lookup_size(d, dull)
    return d, size


def lookup_size(delta: float, dull: dict) -> str:
    """
    :type delta: float
    :type dull: dict, a dictionary of small, medium, large thresholds.
    """
    delta = abs(delta)
    if delta < dull['small']:
        return 'negligible'
    if dull['small'] <= delta < dull['medium']:
        return 'small'
    if dull['medium'] <= delta < dull['large']:
        return 'medium'
    if delta >= dull['large']:
        return 'large'


def runs(lst):
    """Iterator, chunks repeated values"""
    for j, two in enumerate(lst):
        if j == 0:
            one, i = two, 0
        if one != two:
            yield j - i, one
            i = j
        one = two
    yield j - i + 1, two


In [42]:
cliffsDelta(aa,hh)[1]

'large'

In [43]:
cliff_result={}
for key in a_auc_map.keys():
    kk = key.split("defect_")[1]
    print(kk)
    for hkey in h_auc_map.keys():
        if kk in hkey:
            print(hkey)
            aa = a_auc_map[key]
            hh = h_auc_map[hkey]

            cliff_result[hkey] =             cliffsDelta(aa,hh)[1]



camel_1_2
H_camel_1_2_xgboost
H_camel_1_2_randomforest
H_camel_1_2_lr
eclipse_2_0
H_eclipse_2_0_xgboost
H_eclipse_2_0_randomforest
H_eclipse_2_0_lr
eclipse_3_0
H_eclipse_3_0_xgboost
H_eclipse_3_0_randomforest
H_eclipse_3_0_lr
prop_2
H_prop_2_xgboost
H_prop_2_randomforest
H_prop_2_lr
xalan_2_6
H_xalan_2_6_xgboost
H_xalan_2_6_randomforest
H_xalan_2_6_lr


In [44]:
cliff_result

{'H_camel_1_2_xgboost': 'large',
 'H_camel_1_2_randomforest': 'small',
 'H_camel_1_2_lr': 'small',
 'H_eclipse_2_0_xgboost': 'medium',
 'H_eclipse_2_0_randomforest': 'large',
 'H_eclipse_2_0_lr': 'large',
 'H_eclipse_3_0_xgboost': 'medium',
 'H_eclipse_3_0_randomforest': 'large',
 'H_eclipse_3_0_lr': 'large',
 'H_prop_2_xgboost': 'negligible',
 'H_prop_2_randomforest': 'large',
 'H_prop_2_lr': 'large',
 'H_xalan_2_6_xgboost': 'negligible',
 'H_xalan_2_6_randomforest': 'small',
 'H_xalan_2_6_lr': 'large'}